## Парсер сайта citilink.ru (категория смартфоны)

Это парсер я написал за 3,5 часа, учитывая перерыв на чай и скролинг ленты Фейсбука

In [1]:
#начинаем с импорта библиотек
import requests
from requests.auth import HTTPProxyAuth # для проксей
from  bs4 import BeautifulSoup as bs
import csv
from urllib.request import urljoin
from tqdm.notebook import tqdm
from collections import deque
import pandas as pd

In [2]:
# Задаем глобальные параметры
headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'
          }
domen = 'https://www.citilink.ru'
url='https://www.citilink.ru/catalog/mobile/smartfony/'
filename = 'citilink_product_links.csv'

In [39]:
#формируем список страниц для обхода (пагинацию в категории)
pages = get_pagination(url) #запуск пагинатора.на одной итерации обрабатывается одна категория 
#сбор продуктовых урлов с грида        
temp_plinks=[]
for gl in pages:
    gpl = get_product_links(gl, headers, domen)
    temp_plinks.append(gpl)
#выпрямляем список продуктовых урлов    
product_links=[]
for sublist in temp_plinks:
    for item in sublist:
        product_links.append(item)   
save_products(product_links)
print('DONE!')

DONE!


In [8]:
#  блок формирования урлов пагинации
def get_pagination(url):
    pagi=[]
    pagi.append(url)
    for i in range(2,6):
        u = str(url)+f'?/p={i}'
        pagi.append(u)
    return pagi

In [37]:
# Функция сбора ссылок на целевые страницы
def get_product_links(url, headers, domen):
    links = []
    session = requests.Session()
    r = session.get(url, headers=headers)
    soup = bs(r.text, 'lxml')
    urls=set()
    if url:
        ul = soup.find_all('div', class_='ProductCardVertical__description')
        for k in ul:
            href = k.find('a', href=True)['href']
            urls.add(href)
        for i in urls:
            link  = urljoin(domen,i)
            links.append(link)
    return links

In [33]:
def save_products (result):
    with open(filename, 'w', encoding = 'utf-8') as outfile:
        spamwriter = csv.writer(outfile, lineterminator='\n')
        spamwriter.writerow(['Product_Link']) 
        for i in result:
            spamwriter.writerow([i])
            
# Функция загрузки файла со ссылками на целевые страницы
def open_csv(filename):
    results=[]
    with open(filename, 'r') as data:
        data.readline().rstrip()
        for line in data:
            results.append(line[:-1])
    return results

def save_product_info(products):
    with open('products.csv', 'w', encoding = 'utf-8') as outfile:
        spamwriter = csv.DictWriter(outfile, fieldnames=get_field_name(products), lineterminator='\n')    
        spamwriter.writeheader() #записывет заголовки столбцов равные ключам словаря
        spamwriter.writerows(products)
        
# Вспом.функция для сбора названия полей с словарях с продуктами для формирования заголовка в CSV (Без сортировки)
def get_field_name(products):
    field_names = set()
    for i in products:
        for k in i:
            field_names.add(k)
    return field_names

def get_visited_urls():
    try:
        file = 'products.csv'
        if file:
            df = pd.read_csv(file, encoding = 'utf-8')
            urls_done = df['5url'].tolist()
            visited = set(urls_done)
            return visited, df
    except FileNotFoundError:
        df=pd.DataFrame()
        visited=set()
        return visited, df # если файла не существует, товернем путой сет и список

    
def add_previous_part(df):
    if df.shape[0] != 0:
        file = 'products.csv'
        df.to_csv(file, mode='a', index=False, encoding = 'utf-8', header=False)
    else:
        pass   

In [34]:
# ВЕРСИЯ 2 С ПРОМЕЖУТОЧНЫМ СОХРАНЕНИЕМ
def get_info2(headers, filename):
    global df
    results = open_csv(filename)
    queue = deque()
    queue.extend(results)
    visited = get_visited_urls()[0]
    df = get_visited_urls()[1]
    session = requests.Session()
    products = [] 
    while tqdm(queue, desc='Getting info from product page url', leave=False):   
        url = queue.popleft()
        if url in visited:
            continue # начнет цикл while заново     
        params={} # словарь для занесения параметров//можно в конце цикла все добавить как в кинопоиске без создания словаря
        r = session.get(url, headers=headers)
        soup = bs(r.text, 'lxml')
        if soup:
            visited.add(url) #добавляем в посещенные, только когда получим суп. т.е. страница доступна
            try:
                a=soup.find('h1', class_='Heading Heading_level_1 ProductHeader__title').text.strip()
                if a:
                    params['1_title']=a
                b=soup.find('div', class_='ProductHeader__product-id').text
                if b:
                    params['2_articul']=b.lstrip('\n                    Код товара:').strip()
                c=soup.find('span', class_='ProductHeader__price-default_current-price').text
                if c:
                    params['3_price']=c.lstrip('\n                                    ').strip()
                d=soup.find('span', class_='IconWithCount__count js--IconWithCount__count').text
                if d:
                    params['4_rating']=d.lstrip('\n           ').strip()
                params['5url']=url
                ###############################################
                k1=[]
                k2=[]
                dict1={}
                x=soup.find('div', class_='Specifications').find_all('div', class_='Specifications__column Specifications__column_name')
                #print(x)
                for i in x:
                    k1.append(i.text.lstrip('\n                    ').strip())
                y=soup.find('div', class_='Specifications').find_all('div', class_='Specifications__column Specifications__column_value')
                for i in y:
                     k2.append(i.text.lstrip('\n                    ').strip())
                dict1 = dict(zip(k1, k2))
                params.update(dict1)
                products.append(params)
            except:
                products.append(params)
                print('Часть тегов не обнаружена. Двигаюсь дальше')
                pass
        else:
            print('Проблемы с супом')
            save_product_info(products) # промежуточное сохранение на случай если суп не обнаружен
            return False
    save_product_info(products) #  для сохранения в CSV
    add_previous_part(df) # перезапись предыдущих результатов в случае перезапуска парсера
    print('DONE!')

In [37]:
# Запуск самого парсера
get_info2(headers, filename)

DONE!


In [38]:
import pandas as pd
df=pd.read_csv('products.csv', encoding='utf-8')
df.sort_index(inplace=True, axis=1)
df.head(10)

,1_title,2_articul,3_price,4_rating,5url,Время работы в режиме ожидания (max),Время работы в режиме разговора (max),Дисплей,Объем встроенной памяти,Объем оперативной памяти,Операционная система,Процессор,Разрешение дисплея
0,"Смартфон REALME C3 32Gb, серый",1378921,8690,4.7,https://www.citilink.ru/catalog/mobile/cell_ph...,727.7 ч,43.9 ч,"6.5"", LCD",32 ГБ,3 ГБ,Android 10,"MediaTek Helio G70, 2000МГц, 8-ми ядерный",1600x720
1,"Смартфон ZTE Blade 20 Smart 128Gb, черный гранат",1199695,13 490,4.6,https://www.citilink.ru/catalog/mobile/cell_ph...,833 ч,50 ч,"6.49"", IPS",128 ГБ,4 ГБ,Android 9.0,"MediaTek Helio P60, 2000МГц, 8-ми ядерный",1560x720
2,"Смартфон XIAOMI Poco X3 128Gb, синий",1426033,19 490,5.0,https://www.citilink.ru/catalog/mobile/cell_ph...,NaN,NaN,"6.67"", IPS",128 ГБ,6 ГБ,Android 10,"Qualcomm Snapdragon 732G, 2300МГц, 8-ми ядерный",2400x1080
3,"Смартфон XIAOMI Redmi Note 9 Pro 6/128Gb, зел...",1380942,20 950,4.9,https://www.citilink.ru/catalog/mobile/cell_ph...,NaN,33 ч,"6.67"", IPS",128 ГБ,6 ГБ,Android 10.0,"Qualcomm Snapdragon 720G, 2300МГц, 8-ми ядерный",2400x1080
4,"Смартфон SAMSUNG Galaxy M21 64Gb, SM-M215F, ...",1369917,16 350,4.7,https://www.citilink.ru/catalog/mobile/cell_ph...,NaN,49 ч,"6.4"", Super AMOLED",64 ГБ,4 ГБ,Android 10,"Exynos 9611, 2300МГц, 8-ми ядерный",2340x1080
5,"Смартфон XIAOMI Redmi Note 9 128Gb, серый",1385232,15 880,4.7,https://www.citilink.ru/catalog/mobile/cell_ph...,NaN,NaN,"6.53"", IPS",128 ГБ,4 ГБ,Android 10,"Mediatek Helio G85, 2000МГц, 8-ми ядерный",2340x1080
6,"Смартфон HUAWEI Y6P 64Gb, черный",1380404,10 490,4.9,https://www.citilink.ru/catalog/mobile/cell_ph...,NaN,NaN,"6.3"", IPS",64 ГБ,3 ГБ,Android 10 HMS,"MediaTek Helio P22, 2000МГц, 8-ми ядерный",1600x720
7,"Смартфон SAMSUNG Galaxy A71 128Gb, SM-A715F, ...",1211946,28 710,4.1,https://www.citilink.ru/catalog/mobile/cell_ph...,NaN,42 ч,"6.7"", Super AMOLED",128 ГБ,6 ГБ,Android 10,"Qualcomm Snapdragon 730, 2200МГц, 8-ми ядерный",2400x1080
8,"Смартфон XIAOMI Redmi Note 9 Pro 6/128Gb, белый",1380952,20 950,4.7,https://www.citilink.ru/catalog/mobile/cell_ph...,NaN,33 ч,"6.67"", IPS",128 ГБ,6 ГБ,Android 10.0,"Qualcomm Snapdragon 720G, 2300МГц, 8-ми ядерный",2400x1080
9,"Смартфон HONOR 9X 128Gb, черный",1189767,14 790,4.9,https://www.citilink.ru/catalog/mobile/cell_ph...,711 ч,28 ч,"6.59"", LTPS",128 ГБ,4 ГБ,Android 9.0,"Hisilicon Kirin 710F, 2200МГц, 8-ми ядерный",2340x1080
